# Preparing model training

In [1]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\DL_Projects\\2025-projects\\End-to-End-Industrial-Grade-DL-Project\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\DL_Projects\\2025-projects\\End-to-End-Industrial-Grade-DL-Project'

In [4]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
# tf.compat.v1.enable_eager_execution()
print("Eager execution enabled:", tf.executing_eagerly())
a = tf.constant([1.0, 2.0, 3.0])
print(a.numpy())
a = tf.constant([1, 2, 3])
print(a.numpy())  # should print [1 2 3]

Eager execution enabled: True
[1. 2. 3.]
[1 2 3]


In [5]:
#preparing training entity
from pathlib import Path 
from dataclasses import dataclass
from typing import List

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_augmentation: bool
    params_image_size: List[int]


@dataclass(frozen=True)
class CallabacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
#preparing configguration manager

from src.CNNClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.CNNClassifier.utils import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    #preparing_callbacks_configurations
    def get_callbacks_config(self) -> CallabacksConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_ckpt_dir)
            ])

        callbacks_config = CallabacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
            )
        return callbacks_config

    #preparing traing configuration
    def get_training_config(self) ->TrainingConfig:
        training = self.config.training #for root_dir, trained_model_path
        prepare_base_model = self.config.prepare_base_model #for updated_base_model_path
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages") #for training data
        params = self.params  #for params

        create_directories([Path(training.root_dir)]) #to create training- dir
        
        trainig_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
            )
        return trainig_config



In [7]:
#prepare my components
# import tensorflow as tf

import os
# import tensorflow as tf
from zipfile import ZipFile
import urllib.request as request
import time


class Callbacks:
    def __init__(self, config: CallabacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_runing_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir = tb_runing_log_dir)
    
    @property
    def _create_chkp_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )

    #define methods to get tensorboard checkpoints
    def get_tb_chkp_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_chkp_callbacks
        ]

In [8]:
#preparing training component
# import tensorflow as tf
import os
from zipfile import ZipFile
import urllib.request as request
import time


class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config

    
    #def func to get updated_base_model
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)


    #performing data generation
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
            )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
            )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
            )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
            )

        if self.config.params_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
                )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
            )
        

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
            )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
            )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
            )

In [9]:
#preparing pipeline
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callbacks_config()    
    callbacks = Callbacks(config = callbacks_config)
    callbacks_list = callbacks.get_tb_chkp_callbacks()

    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list= callbacks_list)

except Exception as e:
    raise e

[2025-10-06 11:35:59,877: INFO: common]: yaml file: configs\config.yaml loaded successfully]
[2025-10-06 11:35:59,881: INFO: common]: yaml file: params.yaml loaded successfully]
[2025-10-06 11:35:59,883: INFO: common]: Directory created at: artifacts]
[2025-10-06 11:35:59,885: INFO: common]: Directory created at: artifacts\callbacks\tensorboard_root_log_dir]
[2025-10-06 11:35:59,885: INFO: common]: Directory created at: artifacts\callbacks\checkpoint_dir]
[2025-10-06 11:35:59,887: INFO: common]: Directory created at: artifacts\training]
[2025-10-06 11:36:00,275: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 318 images belonging to 2 classes.


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.4792 - loss: 12.4648 [2025-10-06 11:39:16,131: WARNING: saving_api]: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
9/9 ━━━━━━━━━━━━━━━━━━━━ 195s 22s/step - accuracy: 0.4790 - loss: 16.5505 - val_accuracy: 0.3906 - val_loss: 15.9265
Epoch 2/2
1/9 ━━━━━━━━━━━━━━━━━━━━ 2:20 18s/step - accuracy: 0.4688 - loss: 13.7898

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


9/9 ━━━━━━━━━━━━━━━━━━━━ 508s 61s/step - accuracy: 0.4688 - loss: 13.7898 - val_accuracy: 0.6094 - val_loss: 17.0951
[2025-10-06 11:47:44,329: WARNING: saving_api]: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
